<a href="https://colab.research.google.com/github/brianp0513/NLP-project/blob/main/LSTM_NLP_Project_version4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM with custom text preprocessing



Failure note:

I tried to use Countvecter for keras, but it did not work.
Keras seems to work with own builtin text transform function.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf
%matplotlib inline
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
# import seaborn as sns
import time
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
np.random.seed(1234)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
class LSTMversionFour:

    def __init__(self, numberOfDFiles, numberOfNDFiles, data_path_d="reddit_depression",
                 data_path_nd="reddit_non_depression"):

        self.data_path_d = data_path_d
        self.data_path_nd = data_path_nd
        self.data_path_d_test = "reddit_depression_testset"
        self.df = pd.DataFrame(columns=['text', 'depression'])
        self.numberOfDFiles = numberOfDFiles
        self.numberOfNDFiles = numberOfNDFiles
        self.depressionClass = 1
        self.nonDepressionClass = 0
        self.classifier = MultinomialNB()
        self.counts = 0
        self.test_counts= 0
        self.featnames = 0
        self.length_of_maximum_document = 0

    def preprocessing(self):

        self.checkfilesCounts(self.data_path_d)
        self.checkfilesCounts(self.data_path_nd)
        self.df = self.getTextFromFiles(self.df, self.data_path_d, self.depressionClass, self.numberOfDFiles)
        self.df = self.getTextFromFiles(self.df, self.data_path_nd, self.nonDepressionClass, self.numberOfNDFiles)
        self.dataPreprocessingForX(self.df, 'text')
        self.dataPreprocessingForY(self.df, 'depression')
        X = self.df['text'].values
        Y = self.df['depression'].values
        indices = np.arange(len(X))
        X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, indices, test_size=0.15)
        return X_train, X_test, Y_train, Y_test, idx_train, idx_test


    def getTextFromFiles(self, df, data_path, zeroOrOne , limit):
        """Return Data Frame """

        for file in os.listdir(data_path)[:limit]:
            with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
                file1 = file1.read()
                df = df.append({'text': file1, 'depression': int(zeroOrOne)}, ignore_index=True)

        return df



    def dataPreprocessingForX(self, df, columnName1):

        df[columnName1] = df[columnName1].map(lambda text: text.lower())
        df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
        stop_words = set(nltk.corpus.stopwords.words('english'))
        df[columnName1] = df[columnName1].map(lambda tokens: [w for w in tokens if not w in stop_words])
        df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
        df[columnName1] = df[columnName1].map(lambda text: re.sub('[^A-Za-z]+', ' ', text))
        df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
        lemmatizer = WordNetLemmatizer()
        df[columnName1] = df[columnName1].map(lambda text: [lemmatizer.lemmatize(i) for i in text])
        self.length_of_maximum_document = df[columnName1].map(lambda listOfwords: len(listOfwords)).max()
        df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
        # lenght_of_maximum_document

    def dataPreprocessingForY(self, df, columnName2):

        df[columnName2] = df[columnName2].astype('int32')


    def checkfilesCounts(self, data_path):
        print("files Counts in "+ data_path +": ", len(os.listdir(data_path)))

    def fitandPredict(self, X_train, Y_train, X_test):

 
        # predictions = self.classifier.predict(self.test_counts)
        # print(self.counts.shape)
        # sequences_matrix = self.counts.toarray()
        # sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
        # test_sequences_matrix = self.test_counts.toarray()

        # sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
        # 'input_dim' = the vocab size that we will choose. In other words it is the number of unique words in the vocab.
        # 'output_dim' = the number of dimensions we wish to embed into. Each word will be represented by a vector of this much dimensions.
        # 'input_length' = lenght of the maximum document. which is stored in maxlen variable in our case.
        # input_dim = len(self.featnames)
        # output_dim = 10
        # input_length = self.length_of_maximum_document

        max_words = 1000
        max_len = 150

        tokenizer = Tokenizer(num_words=max_words)
        tokenizer.fit_on_texts(X_train)
        sequences = tokenizer.texts_to_sequences(X_train)
        sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

        test_sequences = tokenizer.texts_to_sequences(X_test)
        test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

        # print("input_dim", len(self.featnames))
        # print("output_dim",10)
        # print("input_length", self.length_of_maximum_document)
        # print("sequences_matrix.shape", sequences_matrix.shape)
        # print("sequences_matrix.shape", test_sequences_matrix.shape)

        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Embedding(input_dim = max_words, output_dim = 8, input_length= max_len))
        # model.add(tf.keras.layers.Embedding(max_words,50))
        model.add(tf.keras.layers.LSTM(64))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(256))
        # model.add(tf.keras.layers.Activation('relu'))
        # model.add(tf.keras.layers.Dropout(0.2))
        # model.add(tf.keras.layers.Dense(80))
        model.add(tf.keras.layers.Activation('relu'))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(1))
        model.add(tf.keras.layers.Activation('sigmoid'))

        model.build(input_shape=sequences_matrix.shape)
        
        # model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
        opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(loss='binary_crossentropy',optimizer= opt,metrics=['accuracy'])
        model.summary()
        model.fit(sequences_matrix, Y_train.astype(float), batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
        accr = model.evaluate(test_sequences_matrix,Y_test.astype(float))
        print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

        # print(predictions)


In [19]:
# obj = LSTMversionFour(numberOfDFiles =1000, numberOfNDFiles=500, data_path_d="reddit_depression",
#                      data_path_nd="reddit_non_depression")
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
obj = LSTMversionFour(numberOfDFiles =500, numberOfNDFiles=500, data_path_d=data_path_d,
                     data_path_nd=data_path_nd)
X_train, X_test, Y_train, Y_test, idx_train, idx_test = obj.preprocessing()
predictions = obj.fitandPredict( X_train, Y_train, X_test)

files Counts in /content/drive/My Drive/NLP Team/code/kerasData/reddit_depression:  1293
files Counts in /content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression:  548
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 8)            8000      
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                18688     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropo